# [Soynlp] 학습 기반 토크나이저
- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비지도 학습으로 단어 토큰화 -> 데이터에 자주 등장하는 단어들을 단어로 분석
- 내부적으로 단어 점수 표로 동작

In [5]:
from konlpy.tag import Okt

In [6]:
tokenizer = Okt()

print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다."))

# 형태소 분석 시 매개변수 stem, norm 둘 다 default는 False 
print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다.", stem = True)) # 어간
print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다.", norm = True)) # 정규화 

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '이다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다', '.']


In [7]:
# [soynlp] 사용 -> 말뭉치 데이터셋 
from urllib.request import urlretrieve
import os

In [21]:
filename = "../data/text_data.txt"

In [22]:
### ===> 학습 데이터 처리
from soynlp import DoublespaceLineCorpus # 한개로 통합된 문서 데이터를 분리하기 위함 
from soynlp.word import WordExtractor # 단어 추출 

In [23]:
### ===> 훈련 데이터 문서 분리
corpus = DoublespaceLineCorpus(filename)
print(f"훈련 데이터 문서 : {len(corpus)}개")

훈련 데이터 문서 : 30091개


In [24]:
# [주의] 실행 시 오래걸려서 주석으로 바꿔둠

### ===> SoyNLP 학습 진행
word_extractor = WordExtractor()

# 학습 진행하며 단어별 점수
word_extractor.train(sents=corpus)

# 단어별 점수표 추출
word_score_table = word_extractor.extract()



training was done. used memory 1.196 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [25]:
# 단어별 점수표 확인
for idx, key in enumerate(iterable=word_score_table):
    print(f'[{idx}]-{key}')
    if idx == 30:
        break

[0]-길
[1]-걷
[2]-애
[3]-알
[4]-팁
[5]-덩
[6]-옴
[7]-쿄
[8]-꼴
[9]-참
[10]-챌
[11]-낸
[12]-워
[13]-던
[14]-벅
[15]-넬
[16]-팅
[17]-찬
[18]-뇌
[19]-래
[20]-몸
[21]-덤
[22]-복
[23]-농
[24]-솜
[25]-깼
[26]-같
[27]-뺏
[28]-덜
[29]-핼
[30]-심


In [26]:
### ===> 응집 확률(cohesion probablity) : 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
# - 원리 : 문자열을 문자 단위로 분리, 왼쪽부터 순서대로 문자를 추가
#         각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산 / 누적곱 한 값
# - 값이 높을 수록 : 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성 높음

In [27]:
word_score_table['바'].cohesion_forward

0

In [28]:
word_score_table['바다'].cohesion_forward

0.06393648140409527

In [29]:
word_score_table['바다에'].cohesion_forward

0.11518621707955429

In [30]:
### ===> SOYNLP의 L tokenzer
# - 띄어쓰기 단위로 나누 어절 토큰 : L 토큰 + R 토큰
#   (예 : '공원에' => '공원' + '에', '공부하는' => '공부 + 하는')
# - 분리 기준 : 점수가 가장 높은 L 토큰을 찾아내는 원리

In [31]:
from soynlp.tokenizer import LTokenizer

# 토큰으로 쪼개기 위한 L토큰 
scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

l_tokenizer = LTokenizer(scores = scores)
l_tokenizer.tokenize('국제사회와 우리의 노력들로 범죄를 척결하자', flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [32]:
### ===> 최대 점수 토크나이저
# - 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저
# - 띄어쓰기가 되어 있지 않은 묹아을 넣어서 점수를 통해 토큰화 된 결과

from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)       # MaxScoreTokenizer는 학습된 데이터셋을 바탕으로 하는 것
maxscore_tokenizer.tokenize('국제사회와우리의노력들로범죄를척결하자')

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

In [33]:
### SOYNLP를 이용한 반복되는 문자 정제
# - ㅋㅋ, ㅎㅎ 등의 이모티콘의 경우 불필요하게 연속되는 경우 많음
# - ㅋㅋ, ㅋㅋㅋ, ㅋㅋㅋㅋ와 같은 경우를 모두 서로 다른 단어로 처리하는 것은 불필요
# >>> 반복되는 것은 하나로 정규화

In [35]:
from soynlp.normalizer import *

print(emoticon_normalize('앜ㅋㅋㅋ영화존잼쓰ㅠㅠㅠㅠ', num_repeats=1))
print(emoticon_normalize('앜ㅋㅋㅋ영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋ영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋ영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋ영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠ', num_repeats=3))

아ㅋ화존잼쓰ㅠ
아ㅋㅋ화존잼쓰ㅠㅠ
아ㅋㅋ화존잼쓰ㅠㅠ
아ㅋㅋ화존잼쓰ㅠㅠ
아ㅋㅋㅋ화존잼쓰ㅠㅠㅠ


In [49]:
print(repeat_normalize('와하하하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하핫', num_repeats=2))
print(repeat_normalize('와하하핫', num_repeats=2))
print(repeat_normalize('와하핫', num_repeats=2))

print(repeat_normalize('앜^^^^^^^^^^^^^^^^^^', num_repeats=1))            # ^는 이모티콘으로 인식하지 않음, 구두점에서 제거해야


와하하핫
와하하핫
와하하핫
와하하하핫
와하하핫
와하핫
앜^^^^^^^^^^^^^^^^^^


In [57]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [60]:
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다.')

/Users/sml/anaconda3/envs/Torch_NLP38/lib/python3.8/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [61]:
# 형태소 분석기에 사전 추가
twitter.add_dictionary('은경이','Noun')

In [62]:
twitter.morphs('은경이는 사무실로 갔습니다.')

['은경이', '는', '사무실', '로', '갔습니다', '.']